# PipelineIQ Incremental Processing

This notebook processes new and updated use cases incrementally using watermark-based loading.

**Pipeline Flow:**
1. Load only new/changed records from source (based on `piq_last_updated` watermark)
2. Apply AI enrichments sequentially
3. MERGE results into target table

**Design: Minimized Column Storage**
- The `pipelineiq` table stores only essential columns:
  - `usecase_id` (primary key)
  - `last_modified_date` (from source, to detect updates)
  - `piq_last_updated` (watermark, set to CURRENT_DATE() when processed)
  - `competition_string` (needed for AI inference)
  - All AI-generated fields (context, confidence scores, recommendations, etc.)
- For reporting, join back to `main.gtm_data.core_usecase_curated` on `usecase_id` to get source fields like `usecase_name`, `account_name`, `stage`, etc.
- This minimizes storage and focuses the pipeline on AI enrichment only


In [0]:
# =============================================================================
# CONFIGURATION
# =============================================================================

CONFIG = {
    # Filter Criteria
    "min_monthly_dbus": 5000,
    "excluded_stages": ['U1', 'Lost', 'Closed', 'Disqualified'],
    "target_live_date_months_ahead": 6,
    "last_modified_months_back": -6,
    
    # AI Model Settings
    "ai_model": "databricks-gpt-oss-20b",
    "temperature": 0.0,
    "top_p": 0.1,
    
    # Token Limits
    "max_tokens_summarize": 100,
    "max_tokens_oneliner": 50,
    "max_tokens_confidence": 200,
    "max_tokens_classification": 20,
    "max_tokens_recommendation": 200,
    
    # Confidence Thresholds
    "confidence_high_threshold": 75,
    "confidence_medium_threshold": 45,
    
    # MEDDPICC Weights
    "meddpicc_weights": {
        "pain": 0.25,
        "champion": 0.20,
        "implementation_plan": 0.20,
        "decision_process": 0.10,
        "urgency": 0.10,
        "competition_awareness": 0.05,
        "measurable_impact": 0.02,
        "major_blockers": -0.08
    }
}

# Use Case Type Categories
USECASE_TYPE_CATEGORIES = [
    'Machine Learning', 'Generative AI', 'Data Warehousing', 
    'Migration AI', 'Migration Unity Catalog', 'Migration DWH', 
    'Migration ETL', 'Migration Streaming', 'Migration BI',
    'BI', 'ETL', 'Governance', 'Streaming', 'Ingestion', 'Platform', 'Other'
]

# =============================================================================
# WIDGETS FOR PARAMETERIZATION
# =============================================================================

# Create widgets for catalog and schema
dbutils.widgets.text("catalog", "users", "Target Catalog")
dbutils.widgets.text("schema", "sam_lecorre", "Target Schema")

# Get widget values
TARGET_CATALOG = dbutils.widgets.get("catalog")
TARGET_SCHEMA = dbutils.widgets.get("schema")
TARGET_TABLE = f"{TARGET_CATALOG}.{TARGET_SCHEMA}.pipelineiq"
SOURCE_TABLE = "main.gtm_data.core_usecase_curated"

print(f"Target Catalog: {TARGET_CATALOG}")
print(f"Target Schema: {TARGET_SCHEMA}")
print(f"Target Table: {TARGET_TABLE}")
print(f"Source Table: {SOURCE_TABLE}")

# =============================================================================
# SETUP & VALIDATION
# =============================================================================

# Set Spark configurations for performance
spark.conf.set("spark.sql.shuffle.partitions", "64")
spark.conf.set("spark.databricks.execution.timeout", "14400")

# Check if target table exists
table_exists = spark.catalog.tableExists(TARGET_TABLE)

if not table_exists:
    print(f"⚠️  Target table {TARGET_TABLE} does not exist. Creating empty table...")
    
    # Create empty target table with schema from source
    # We'll add the AI enrichment columns in the first merge
    spark.sql(f"""
        CREATE TABLE IF NOT EXISTS {TARGET_TABLE}
        (
        -- Core identifiers and timestamps
        usecase_id STRING COMMENT 'Primary key - use case identifier from source system',
        last_modified_date DATE COMMENT 'Last modified date from source system (used to detect updates)',
        piq_last_updated DATE COMMENT 'Watermark column - date when this record was last processed by PipelineIQ',
        
        -- Fields needed for AI inference
        competition_string STRING COMMENT 'Competition information from source (used for AI inference)',
        num_of_blockers BIGINT COMMENT 'Number of blockers from source',
        
        -- AI enriched fields
        context STRING COMMENT 'AI-generated summary of BDR context and business impact for the use case',
        one_liner STRING COMMENT 'AI-generated concise summary of the use case',
        next_steps_summary STRING COMMENT 'AI-generated summary of next steps for the use case',
        implementation_notes_summary STRING COMMENT 'AI-generated summary of implementation notes',
        blockers_summary STRING COMMENT 'AI-generated summary of blockers',
        usecase_type STRING COMMENT 'AI-classified technical use case category',
        business_usecase_type STRING COMMENT 'AI-generated business use case label (max 5 words)',
        
        -- Confidence scoring
        ai_confidence_score_advanced STRING COMMENT 'Raw AI-generated MEDDPICC confidence assessment output',
        confidence_score INT COMMENT 'AI-extracted confidence score (0-100) for successful go-live',
        confidence_level STRING COMMENT 'AI-extracted confidence level (High, Medium, Low)',
        rationale_for_confidence_score STRING COMMENT 'AI-extracted rationale for confidence score',
        pain_score INT COMMENT 'AI-extracted MEDDPICC Pain dimension score (0-10)',
        champion_score INT COMMENT 'AI-extracted MEDDPICC Champion dimension score (0-10)',
        implementationplan_score INT COMMENT 'AI-extracted MEDDPICC Implementation Plan dimension score (0-10)',
        decisionprocess_score INT COMMENT 'AI-extracted MEDDPICC Decision Process dimension score (0-10)',
        urgency_score INT COMMENT 'AI-extracted MEDDPICC Urgency dimension score (0-10)',
        competitionawareness_score INT COMMENT 'AI-extracted MEDDPICC Competition Awareness dimension score (0-10)',
        majorblockers_score INT COMMENT 'AI-extracted MEDDPICC Major Blockers dimension score (0-10)',
        measurableimpact_score INT COMMENT 'AI-extracted MEDDPICC Measurable Impact dimension score (0-10)',
        confidence_level_normalized STRING COMMENT 'Normalized confidence level (High, Medium, Low, Not computed)',
        
        -- Recommendations and classifications
        next_best_action_recommendation STRING COMMENT 'AI-generated next best action recommendation for account team',
        likely_to_slip BOOLEAN COMMENT 'Flag if use case is likely to slip (true/false)',
        slippage_category STRING COMMENT 'AI-classified primary reason for slippage',
        can_be_accelerated BOOLEAN COMMENT 'Flag if use case can be accelerated (true/false)',
        accel_category STRING COMMENT 'AI-classified best acceleration lever'
        )
        USING DELTA
        COMMENT 'PipelineIQ AI-enriched use case insights with minimized storage footprint'
        TBLPROPERTIES (
        'delta.enableChangeDataFeed' = 'true'
        )
    """)
    
    print(f"✅ Created empty table {TARGET_TABLE}")
    watermark_date = "1900-01-01"
else:
    print(f"✅ Target table {TARGET_TABLE} exists")
    
    # Query watermark - use piq_last_updated as the watermark column
    watermark_result = spark.sql(f"""
        SELECT COALESCE(MAX(piq_last_updated), '1900-01-01') AS max_date
        FROM {TARGET_TABLE}
    """).first()
    
    watermark_date = str(watermark_result['max_date'])
    print(f"📅 Watermark date (piq_last_updated): {watermark_date}")

# Store watermark as temp variable for SQL
spark.sql(f"""DECLARE OR REPLACE VARIABLE watermark_date STRING DEFAULT '{watermark_date}'""")

# Store config values as SQL variables
spark.sql(f"""DECLARE OR REPLACE VARIABLE min_dbus INT DEFAULT {CONFIG['min_monthly_dbus']}""")
spark.sql(f"""DECLARE OR REPLACE VARIABLE target_months INT DEFAULT {CONFIG['target_live_date_months_ahead']}""")
spark.sql(f"""DECLARE OR REPLACE VARIABLE last_modified_months INT DEFAULT {CONFIG['last_modified_months_back']}""")

print(f"\n✅ Setup complete. Ready to process incremental data.")


In [0]:
%sql
-- Load and filter new/updated use cases based on watermark
-- Only load records where source last_modified_date > our piq_last_updated watermark
CREATE OR REPLACE TEMPORARY VIEW filtered_usecases AS
SELECT 
  usecase_id,
  last_modified_date,
  
  -- Fields needed for AI inference
  sdr_bdr_notes,
  business_impact,
  business_outcome_description,
  ae_authority,
  ae_money_budget,
  usecase_name,
  usecase_description,
  demand_plan_stage_next_steps,
  implementation_notes,
  COALESCE(num_of_blockers, 0) AS num_of_blockers,
  last_modified_blocker_category,
  last_modified_blocker_comment,
  competition_string
  
FROM main.gtm_data.core_usecase_curated
WHERE 
  last_modified_date > watermark_date
  AND estimated_monthly_dollar_dbus >= min_dbus
  AND stage NOT IN ('U1', 'Lost', 'Closed', 'Disqualified')
  AND target_live_date BETWEEN CURRENT_DATE() AND ADD_MONTHS(CURRENT_DATE(), target_months)
  AND last_modified_date >= ADD_MONTHS(CURRENT_DATE(), last_modified_months)
ORDER BY estimated_monthly_dollar_dbus DESC;

-- Display count and sample
SELECT 
  COUNT(*) AS new_usecases_count,
  MIN(last_modified_date) AS oldest_modified,
  MAX(last_modified_date) AS newest_modified
FROM filtered_usecases;


In [0]:
%sql
-- Generate contextual summaries using AI
CREATE OR REPLACE TEMPORARY VIEW ai_context AS
SELECT 
  usecase_id,
  CASE 
    WHEN (sdr_bdr_notes IS NOT NULL AND LENGTH(TRIM(sdr_bdr_notes)) > 0)
      OR (business_impact IS NOT NULL AND LENGTH(TRIM(business_impact)) > 0)
      OR (business_outcome_description IS NOT NULL AND LENGTH(TRIM(business_outcome_description)) > 0)
      OR (ae_authority IS NOT NULL AND LENGTH(TRIM(ae_authority)) > 0)
      OR (ae_money_budget IS NOT NULL AND LENGTH(TRIM(ae_money_budget)) > 0)
    THEN
      ai_summarize(
        CONCAT(
          'Summarize the following context and business impact into a concise paragraph (max 4 sentences). Focus on the main challenge, expected business value, the main type of contact (e.g., Executive sponsor, technical decision maker, Economic decision maker, etc), and whether funding is available or approved. Context: ',
          COALESCE(TRIM(LEFT(regexp_replace(sdr_bdr_notes, '<[^>]+>', ''), 2000)), 'No context provided. '),
          'Business Impact: ', COALESCE(TRIM(business_impact), 'No business impact provided. '),
          'Business Outcome: ', COALESCE(TRIM(business_outcome_description), 'No business outcome provided. '),
          'Main Contact Type: ', COALESCE(TRIM(ae_authority), 'Not specified.'),
          'Funding Status: ', COALESCE(TRIM(ae_money_budget), 'Not specified.')
        ),
        100
      )
    ELSE
      'No context or business impact provided.'
  END AS context
FROM filtered_usecases;

-- Preview
-- SELECT * FROM ai_context LIMIT 5;


In [0]:
%sql
-- Generate one-liner summaries using AI
CREATE OR REPLACE TEMPORARY VIEW ai_oneliner AS
SELECT 
  f.usecase_id,
  CASE
    WHEN (f.usecase_name IS NOT NULL AND LENGTH(TRIM(f.usecase_name)) > 0)
      OR (f.usecase_description IS NOT NULL AND LENGTH(TRIM(f.usecase_description)) > 0)
    THEN
      ai_summarize(
        CONCAT(
          'Generate a short, clear one-line summary (max 50 words) describing the core business objective of this use case. ',
          'Account Context: ', COALESCE(TRIM(c.context), 'No account context'),
          ' Use Case Name: ', COALESCE(TRIM(f.usecase_name), 'Use Case name not provided.'),
          '. Description: ', COALESCE(TRIM(f.usecase_description), 'No description provided.')
        ),
        50
      )
    ELSE
      'No use case name or description provided'
  END AS one_liner
FROM filtered_usecases f
LEFT JOIN ai_context c ON f.usecase_id = c.usecase_id;

-- SELECT * FROM ai_oneliner LIMIT 5;


In [0]:
%sql
-- Generate next steps summaries using AI
CREATE OR REPLACE TEMPORARY VIEW ai_nextsteps AS
SELECT 
  usecase_id,
  CASE 
    WHEN demand_plan_stage_next_steps IS NOT NULL 
         AND LENGTH(TRIM(demand_plan_stage_next_steps)) > 0 THEN
      ai_summarize(
        CONCAT(
          'Summarize the following next steps into a short, action-oriented sentence focusing on immediate priorities: ',
          TRIM(demand_plan_stage_next_steps)
        ),
        100
      )
    ELSE
      'No next steps provided'
  END AS next_steps_summary
FROM filtered_usecases;

-- SELECT * FROM ai_nextsteps LIMIT 5;


In [0]:
%sql
-- Generate implementation notes summaries using AI
CREATE OR REPLACE TEMPORARY VIEW ai_implementation AS
SELECT 
  usecase_id,
  CASE 
    WHEN implementation_notes IS NOT NULL AND LENGTH(TRIM(implementation_notes)) > 0 THEN
      ai_summarize(
        CONCAT(
          'Summarize the following implementation notes into a short, action-oriented paragraph focusing on key steps, milestones, and ownership: ',
          implementation_notes
        ),
        100
      )
    ELSE
      'There is no implementation plan or not specified in Salesforce'
  END AS implementation_notes_summary
FROM filtered_usecases;

-- SELECT * FROM ai_implementation LIMIT 5;


In [0]:
%sql
-- Generate blockers summaries using AI
CREATE OR REPLACE TEMPORARY VIEW ai_blockers AS
SELECT 
  usecase_id,
  CASE 
    WHEN num_of_blockers = 0 THEN 'There are 0 blockers'
    WHEN (last_modified_blocker_category IS NULL OR LENGTH(TRIM(last_modified_blocker_category)) = 0)
      AND (last_modified_blocker_comment IS NULL OR LENGTH(TRIM(last_modified_blocker_comment)) = 0)
      THEN 'Blocker details not provided'
    ELSE CONCAT(
      'There are ', CAST(num_of_blockers AS STRING), ' blockers. ',
      ai_summarize(
        CONCAT(
          'There are ', CAST(num_of_blockers AS STRING), ' blockers. Briefly summarize the main theme or issue based on the latest blocker comment: ',
          CASE 
            WHEN last_modified_blocker_category IS NOT NULL AND LENGTH(TRIM(last_modified_blocker_category)) > 0 
              THEN CONCAT('Category: ', TRIM(last_modified_blocker_category), '. ')
            ELSE ''
          END,
          COALESCE(TRIM(last_modified_blocker_comment), 'No blocker comment provided.')
        ),
        50
      )
    )
  END AS blockers_summary
FROM filtered_usecases;

-- SELECT * FROM ai_blockers LIMIT 5;


In [0]:
%sql
-- Classify use case types using AI
CREATE OR REPLACE TEMPORARY VIEW ai_usecase_types AS
SELECT 
  usecase_id,
  CASE
    WHEN (usecase_name IS NOT NULL AND LENGTH(TRIM(usecase_name)) > 0)
      OR (usecase_description IS NOT NULL AND LENGTH(TRIM(usecase_description)) > 0)
    THEN COALESCE(
      ai_classify(
        concat('Usecase Name: ', coalesce(trim(usecase_name), '.'), ' Description: ', coalesce(trim(usecase_description), '.')),
        array(
          'Machine Learning', 'Generative AI', 'Data Warehousing', 'Migration AI', 'Migration Unity Catalog',
          'Migration DWH', 'Migration ETL', 'Migration Streaming', 'Migration BI', 
          'BI', 'ETL', 'Governance', 'Streaming', 'Ingestion', 'Platform', 'Other'
        )
      ),
      'Other'
    )
    ELSE 'Other'
  END AS usecase_type,
  
  COALESCE(
    ai_query(
      'databricks-gpt-oss-20b',
      CONCAT(
        'Given a use case name and description, return ONLY a concise business use case label (e.g. Fraud Detection, Inventory Forecasting, Customer Segmentation). Output exactly one short phrase, max 5 words, no punctuation or explanation. Usecase Name: ',
        COALESCE(TRIM(usecase_name), 'Unknown'), 
        '. Description: ', COALESCE(TRIM(usecase_description), 'No description')
      ),
      named_struct('max_tokens', 20, 'temperature', 0.0, 'failOnError', false)
    ),
    'Unknown Business Use Case'
  ) AS business_usecase_type
FROM filtered_usecases;

-- SELECT * FROM ai_usecase_types LIMIT 5;


In [0]:
%sql
-- Generate AI confidence scores using MEDDPICC framework
CREATE OR REPLACE TEMPORARY VIEW ai_confidence AS
SELECT
  f.usecase_id,
  COALESCE(
    ai_query(
      'databricks-gpt-oss-20b',
      CONCAT(
        'You are a Databricks Solutions Architect evaluating a customer use case and scoring its likelihood to reach production and generate sustained usage.\n\n',
        'Rate the use case using these MEDDPICC-based dimensions (0–10 each):\n',
        '1. Pain – Clear, urgent business problem?\n',
        '2. Champion – Identified internal advocate?\n',
        '3. Implementation Plan – Concrete next steps and ownership?\n',
        '4. Decision Process – Clear decision path?\n',
        '5. Urgency – Time pressure or deadline?\n',
        '6. Competition Awareness – Known competitors?\n',
        '7. Measurable Impact – Defined success metrics?\n',
        '8. Major Blockers – Significant obstacles (higher score = **more blockers**).\n\n',
        'Rules:\n',
        '- Base scores strictly on available evidence — do not speculate or imagine missing info.\n',
        '- If information is missing, assign a low score (≤3) for that dimension.\n',
        '- Use these weightings: Pain 25%, Champion 20%, Implementation Plan 20%, Decision Process 10%, Urgency 10%, Competition Awareness 5%, Measurable Impact 2%, Major Blockers -8% (deductive).\n',
        '- Compute the total confidence_score as: (Pain×0.25 + Champion×0.20 + ImplementationPlan×0.20 + DecisionProcess×0.10 + Urgency×0.10 + CompetitionAwareness×0.05 + MeasurableImpact×0.02 - MajorBlockers×0.08) × 10.\n',
        '- A perfect use case (10s across positive dimensions, 0 blockers) scores 100.\n',
        '- The total confidence_score must be between 0 and 100, rounded to the nearest multiple of 10.\n\n',
        'Context:\n',
        'Account context: ', COALESCE(ctx.context, 'None'), '\n',
        'Use Case: ', COALESCE(one.one_liner, 'None'), '\n',
        'Next steps: ', COALESCE(ns.next_steps_summary, 'None'), '\n',
        'Implementation notes: ', COALESCE(imp.implementation_notes_summary, 'None'), '\n',
        'Blockers: ', COALESCE(blk.blockers_summary, 'None'), '\n',
        'Competition: ', COALESCE(f.competition_string, 'None'), '\n\n',
        'Return output in **this exact format** (no text before or after):\n\n',
        'CONFIDENCE_SCORE: [0–100 number]\n',
        'LEVEL: [High || Medium || Low]\n',
        'RATIONALE: [Brief explanation, 2–3 sentences max]\n',
        'DIMENSION_SCORES: Pain=x, Champion=x, ImplementationPlan=x, DecisionProcess=x, Urgency=x, CompetitionAwareness=x, MajorBlockers=x, MeasurableImpact=x'
      ),
      named_struct('max_tokens', 200, 'temperature', 0.0, 'top_p', 0.1, 'failOnError', false)
    ),
    'CONFIDENCE_SCORE: 0\nLEVEL: Low\nRATIONALE: No information provided.\nDIMENSION_SCORES: Pain=0, Champion=0, ImplementationPlan=0, DecisionProcess=0, Urgency=0, CompetitionAwareness=0, MajorBlockers=0, MeasurableImpact=0'
  ) AS ai_confidence_score_advanced
FROM filtered_usecases f
LEFT JOIN ai_context ctx ON f.usecase_id = ctx.usecase_id
LEFT JOIN ai_oneliner one ON f.usecase_id = one.usecase_id
LEFT JOIN ai_nextsteps ns ON f.usecase_id = ns.usecase_id
LEFT JOIN ai_implementation imp ON f.usecase_id = imp.usecase_id
LEFT JOIN ai_blockers blk ON f.usecase_id = blk.usecase_id;

SELECT * FROM ai_confidence LIMIT 3;


In [0]:
%sql
-- Extract confidence score components using regex
CREATE OR REPLACE TEMPORARY VIEW parsed_confidence AS
SELECT
  usecase_id,
  ai_confidence_score_advanced,
  
  CAST(regexp_extract(ai_confidence_score_advanced, 'CONFIDENCE_SCORE:\\s*(\\d+)', 1) AS INT) AS confidence_score,
  regexp_extract(ai_confidence_score_advanced, 'LEVEL:\\s*(High|Medium|Low)', 1) AS confidence_level,
  regexp_extract(ai_confidence_score_advanced, 'RATIONALE:\\s*(.+?)\\s*DIMENSION_SCORES:', 1) AS rationale_for_confidence_score,
  
  CAST(regexp_extract(ai_confidence_score_advanced, 'Pain=(\\d+)', 1) AS INT) AS pain_score,
  CAST(regexp_extract(ai_confidence_score_advanced, 'Champion=(\\d+)', 1) AS INT) AS champion_score,
  CAST(regexp_extract(ai_confidence_score_advanced, 'ImplementationPlan=(\\d+)', 1) AS INT) AS implementationplan_score,
  CAST(regexp_extract(ai_confidence_score_advanced, 'DecisionProcess=(\\d+)', 1) AS INT) AS decisionprocess_score,
  CAST(regexp_extract(ai_confidence_score_advanced, 'Urgency=(\\d+)', 1) AS INT) AS urgency_score,
  CAST(regexp_extract(ai_confidence_score_advanced, 'CompetitionAwareness=(\\d+)', 1) AS INT) AS competitionawareness_score,
  CAST(regexp_extract(ai_confidence_score_advanced, 'MajorBlockers=(\\d+)', 1) AS INT) AS majorblockers_score,
  CAST(regexp_extract(ai_confidence_score_advanced, 'MeasurableImpact=(\\d+)', 1) AS INT) AS measurableimpact_score
FROM ai_confidence;

SELECT confidence_score, confidence_level, pain_score, champion_score, implementationplan_score FROM parsed_confidence LIMIT 5;


In [0]:
%sql
-- Apply normalized confidence level thresholds
CREATE OR REPLACE TEMPORARY VIEW normalized_confidence AS
SELECT
  *,
  CASE
    WHEN confidence_score >= 75 THEN 'High'
    WHEN confidence_score >= 45 THEN 'Medium'
    WHEN confidence_score IS NOT NULL THEN 'Low'
    ELSE 'Not computed'
  END AS confidence_level_normalized
FROM parsed_confidence;

SELECT confidence_level_normalized, COUNT(*) AS count FROM normalized_confidence GROUP BY confidence_level_normalized ORDER BY count DESC;


In [0]:
%sql
-- Generate next best action recommendations
CREATE OR REPLACE TEMPORARY VIEW ai_next_best_action AS
SELECT
  nc.usecase_id,
  COALESCE(
    ai_query(
      'databricks-gpt-oss-20b',
      CONCAT(
        'You are a Databricks Solutions Architect helping an account team decide the next best action for a customer account and a specific use case.\n',
        'Inputs:\n',
        'Account context: ', COALESCE(ctx.context, 'None'), '\n',
        'Use Case: ', COALESCE(one.one_liner, 'None'), '\n',
        'Next steps summary: ', COALESCE(ns.next_steps_summary, 'None'), '\n',
        'Implementation notes: ', COALESCE(imp.implementation_notes_summary, 'None'), '\n',
        'Blockers: ', COALESCE(blk.blockers_summary, 'None'), '\n',
        'Competition: ', COALESCE(f.competition_string, 'None'), '\n',
        'MEDDPICC dimension scores: Pain=', COALESCE(CAST(nc.pain_score AS STRING), 'None'),
        ', Champion=', COALESCE(CAST(nc.champion_score AS STRING), 'None'),
        ', ImplementationPlan=', COALESCE(CAST(nc.implementationplan_score AS STRING), 'None'),
        ', DecisionProcess=', COALESCE(CAST(nc.decisionprocess_score AS STRING), 'None'),
        ', Urgency=', COALESCE(CAST(nc.urgency_score AS STRING), 'None'),
        ', CompetitionAwareness=', COALESCE(CAST(nc.competitionawareness_score AS STRING), 'None'),
        ', MajorBlockers=', COALESCE(CAST(nc.majorblockers_score AS STRING), 'None'),
        ', MeasurableImpact=', COALESCE(CAST(nc.measurableimpact_score AS STRING), 'None'), '\n',
        'Overall confidence level: ', COALESCE(nc.confidence_level_normalized, 'None'), '\n\n',
        'Rules:\n',
        '1. Base your recommendation strictly on the evidence provided — do not assume missing information.\n',
        '2. Prioritize actions that maximize the likelihood of a successful go-live and reduce blockers.\n',
        '3. If Pain, Champion, or Implementation Plan is low (<5/10), suggest actions to strengthen these dimensions first.\n',
        '4. If Major Blockers > 5/10, suggest actions to mitigate or remove the blockers.\n',
        '5. If Urgency is high (>7/10), prioritize time-sensitive actions.\n',
        '6. Provide one concise, actionable next step, written as if for the account team to execute within the next week.\n',
        '7. Keep your answer short — 1–2 sentences max.\n',
        '8. Be deterministic: always follow the rules exactly and do not vary wording unnecessarily.\n\n',
        'Return output in this exact format (no extra text):\n\n',
        'NEXT_BEST_ACTION: [Short, actionable recommendation for the account team]\n',
        'RATIONALE: [Brief explanation linking the action to the MEDDPICC scores and blockers, ≤2 sentences]'
      ),
      named_struct('max_tokens', 200, 'temperature', 0.0, 'top_p', 0.1, 'failOnError', false)
    ),
    'NEXT_BEST_ACTION: Review MEDDPICC evidence and address missing information.\nRATIONALE: No sufficient context or scores provided to generate a specific action.'
  ) AS next_best_action_recommendation
FROM normalized_confidence nc
LEFT JOIN filtered_usecases f ON nc.usecase_id = f.usecase_id
LEFT JOIN ai_context ctx ON nc.usecase_id = ctx.usecase_id
LEFT JOIN ai_oneliner one ON nc.usecase_id = one.usecase_id
LEFT JOIN ai_nextsteps ns ON nc.usecase_id = ns.usecase_id
LEFT JOIN ai_implementation imp ON nc.usecase_id = imp.usecase_id
LEFT JOIN ai_blockers blk ON nc.usecase_id = blk.usecase_id;

SELECT * FROM ai_next_best_action LIMIT 3;


In [0]:
%sql
-- Classify slippage and acceleration categories
CREATE OR REPLACE TEMPORARY VIEW ai_slippage_acceleration AS
SELECT 
  nc.usecase_id,
  
  -- Flag use cases likely to slip
  CASE 
    WHEN nc.confidence_level_normalized = 'Medium' THEN true
    ELSE false
  END AS likely_to_slip,
  
  -- Classify slippage category
  CASE
    WHEN nc.confidence_level_normalized = 'Medium'
    THEN COALESCE(
      ai_query(
        'databricks-gpt-oss-20b',
        CONCAT(
          'You are a Databricks Senior Solutions Architect.\n',
          'Classify the PRIMARY reason this use case is likely to slip. Choose exactly ONE from:\n',
          '[Technical, Business, Stakeholder, Budget, Project Timelines, Data, Integration, Partner, Hyperscaler, Competition, Other, External dependencies]\n\n',
          'Decision Rules:\n',
          '- Technical → missing features, blockers, or bugs.\n',
          '- Business → shifting priorities or unclear value.\n',
          '- Stakeholder → low buy-in or sponsor turnover.\n',
          '- Budget → funding or approval delays.\n',
          '- Project Timelines → missed milestones or dependencies.\n',
          '- Data → quality, access, or integration issues.\n',
          '- Integration → tooling or system connectivity issues.\n',
          '- Partner → bandwidth or enablement gaps.\n',
          '- Hyperscaler → regional, quota, or policy constraints.\n',
          '- Competition → market or rival-driven reprioritization.\n',
          '- External dependencies → third-party or customer-side blockers.\n',
          '- Other → none of the above.\n\n',
          'Return exactly ONE of those category names. No punctuation or explanation.\n\n',
          'Inputs:\n',
          'Confidence Score Rationale: ', COALESCE(LEFT(nc.rationale_for_confidence_score, 200), 'N/A'), '\n',
          'Next Step: ', COALESCE(LEFT(nba.next_best_action_recommendation, 200), 'N/A')
        ),
        named_struct('max_tokens', 20, 'temperature', 0.0, 'top_p', 0.1, 'failOnError', false)
      ),
      'Unable to determine slippage category'
    )
    ELSE 'N/A'
  END AS slippage_category,
  
  -- Flag use cases that can be accelerated
  CASE 
    WHEN nc.confidence_level_normalized IN ('Medium', 'High') THEN true
    ELSE false
  END AS can_be_accelerated,
  
  -- Classify acceleration category
  CASE
    WHEN nc.confidence_level_normalized IN ('Medium', 'High') THEN 
      COALESCE(
      ai_query(
        'databricks-gpt-oss-20b',
        CONCAT(
          'You are a Databricks Senior Solutions Architect.\n',
          'Classify the PRIMARY reason this use case could be accelerated.\n',
          'Choose exactly ONE label from:\n',
          '[Implementation Planning, Resource Allocation, Stakeholder Engagement, Technical Optimization, Partnership Leverage, Governance and Support, Innovation and Technology Enablement, Capability Development, Funding, Other]\n\n',
          'Decision rules:\n',
          '- Implementation Planning → clarifying the WHAT/WHEN/WHO (critical path, milestones, owners).\n',
          '- Resource Allocation → people/tools/compute NOW.\n',
          '- Stakeholder Engagement → alignment, cadence, or decisions.\n',
          '- Technical Optimization → performance, architecture, integration tuning.\n',
          '- Innovation and Technology Enablement → templates, automation, AI accelerators.\n',
          '- Governance and Support → steering, dashboards, SLAs, support.\n',
          '- Capability Development → training, upskilling, knowledge transfer.\n',
          '- Partnership Leverage → partner expertise/assets as key lever.\n',
          '- Funding → securing or expediting budget, approvals, resources.\n',
          '- Other → none of the above.\n\n',
          'Return exactly one of the labels above — no punctuation or explanation.\n\n',
          'Context:\n',
          'Confidence Score Rationale: ', COALESCE(LEFT(nc.rationale_for_confidence_score, 200), 'N/A'), '\n',
          'Next Step: ', COALESCE(LEFT(nba.next_best_action_recommendation, 200), 'N/A')
        ),
       named_struct('max_tokens', 20, 'temperature', 0.0, 'top_p', 0.1, 'failOnError', false)
      ),
      'Unable to determine acceleration category'
    )
    ELSE 'N/A'
  END AS accel_category
FROM normalized_confidence nc
LEFT JOIN ai_next_best_action nba ON nc.usecase_id = nba.usecase_id;

SELECT likely_to_slip, slippage_category, can_be_accelerated, accel_category FROM ai_slippage_acceleration LIMIT 5;


In [0]:
%sql
-- Join all AI enrichments - keep only minimal columns
-- Can always re-join to source table on usecase_id for other fields
CREATE OR REPLACE TEMPORARY VIEW enriched_results AS
SELECT 
  -- Core identifiers and timestamps
  base.usecase_id,
  base.last_modified_date,
  CURRENT_DATE() AS piq_last_updated,
  
  -- Fields needed for AI inference (kept for reference)
  base.competition_string,
  base.num_of_blockers,
  -- AI enriched fields
  ctx.context,
  one.one_liner,
  ns.next_steps_summary,
  imp.implementation_notes_summary,
  blk.blockers_summary,
  uct.usecase_type,
  uct.business_usecase_type,
  
  -- Confidence scoring
  nc.ai_confidence_score_advanced,
  nc.confidence_score,
  nc.confidence_level,
  nc.rationale_for_confidence_score,
  nc.pain_score,
  nc.champion_score,
  nc.implementationplan_score,
  nc.decisionprocess_score,
  nc.urgency_score,
  nc.competitionawareness_score,
  nc.majorblockers_score,
  nc.measurableimpact_score,
  nc.confidence_level_normalized,
  
  -- Recommendations and classifications
  nba.next_best_action_recommendation,
  sa.likely_to_slip,
  sa.slippage_category,
  sa.can_be_accelerated,
  sa.accel_category
FROM filtered_usecases base
LEFT JOIN ai_context ctx ON base.usecase_id = ctx.usecase_id
LEFT JOIN ai_oneliner one ON base.usecase_id = one.usecase_id
LEFT JOIN ai_nextsteps ns ON base.usecase_id = ns.usecase_id
LEFT JOIN ai_implementation imp ON base.usecase_id = imp.usecase_id
LEFT JOIN ai_blockers blk ON base.usecase_id = blk.usecase_id
LEFT JOIN ai_usecase_types uct ON base.usecase_id = uct.usecase_id
LEFT JOIN normalized_confidence nc ON base.usecase_id = nc.usecase_id
LEFT JOIN ai_next_best_action nba ON base.usecase_id = nba.usecase_id
LEFT JOIN ai_slippage_acceleration sa ON base.usecase_id = sa.usecase_id;

-- Show count of enriched records
-- SELECT 
--   COUNT(*) AS total_enriched_records,
--   COUNT(context) AS with_context,
--   COUNT(confidence_score) AS with_confidence_score
-- FROM enriched_results;


In [0]:
%sql
-- Merge enriched results into target table
-- Updates when source last_modified_date is newer, always updates piq_last_updated
MERGE INTO IDENTIFIER(:catalog || '.' || :schema || '.pipelineiq') AS target
USING enriched_results AS source
ON target.usecase_id = source.usecase_id
WHEN MATCHED AND source.last_modified_date > target.last_modified_date THEN
  UPDATE SET *
WHEN NOT MATCHED THEN
  INSERT *;


In [0]:
%sql
-- Add column comments for AI-generated fields
ALTER TABLE IDENTIFIER(:catalog || '.' || :schema || '.pipelineiq') ALTER COLUMN usecase_id COMMENT 'Primary key - use case identifier from source system';
ALTER TABLE IDENTIFIER(:catalog || '.' || :schema || '.pipelineiq') ALTER COLUMN last_modified_date COMMENT 'Last modified date from source system (used to detect updates)';
ALTER TABLE IDENTIFIER(:catalog || '.' || :schema || '.pipelineiq') ALTER COLUMN piq_last_updated COMMENT 'Watermark column - date when this record was last processed by PipelineIQ';
ALTER TABLE IDENTIFIER(:catalog || '.' || :schema || '.pipelineiq') ALTER COLUMN competition_string COMMENT 'Competition information from source (used for AI inference)';
ALTER TABLE IDENTIFIER(:catalog || '.' || :schema || '.pipelineiq') ALTER COLUMN context COMMENT 'AI-generated summary of BDR context and business impact for the use case';
ALTER TABLE IDENTIFIER(:catalog || '.' || :schema || '.pipelineiq') ALTER COLUMN one_liner COMMENT 'AI-generated concise summary of the use case';
ALTER TABLE IDENTIFIER(:catalog || '.' || :schema || '.pipelineiq') ALTER COLUMN next_steps_summary COMMENT 'AI-generated summary of next steps for the use case';
ALTER TABLE IDENTIFIER(:catalog || '.' || :schema || '.pipelineiq') ALTER COLUMN implementation_notes_summary COMMENT 'AI-generated summary of implementation notes';
ALTER TABLE IDENTIFIER(:catalog || '.' || :schema || '.pipelineiq') ALTER COLUMN blockers_summary COMMENT 'AI-generated summary of blockers';
ALTER TABLE IDENTIFIER(:catalog || '.' || :schema || '.pipelineiq') ALTER COLUMN usecase_type COMMENT 'AI-classified technical use case category';
ALTER TABLE IDENTIFIER(:catalog || '.' || :schema || '.pipelineiq') ALTER COLUMN business_usecase_type COMMENT 'AI-generated business use case label (max 5 words)';
ALTER TABLE IDENTIFIER(:catalog || '.' || :schema || '.pipelineiq') ALTER COLUMN ai_confidence_score_advanced COMMENT 'Raw AI-generated MEDDPICC confidence assessment output';
ALTER TABLE IDENTIFIER(:catalog || '.' || :schema || '.pipelineiq') ALTER COLUMN confidence_score COMMENT 'AI-extracted confidence score (0-100) for successful go-live';
ALTER TABLE IDENTIFIER(:catalog || '.' || :schema || '.pipelineiq') ALTER COLUMN confidence_level COMMENT 'AI-extracted confidence level (High, Medium, Low)';
ALTER TABLE IDENTIFIER(:catalog || '.' || :schema || '.pipelineiq') ALTER COLUMN rationale_for_confidence_score COMMENT 'AI-extracted rationale for confidence score';
ALTER TABLE IDENTIFIER(:catalog || '.' || :schema || '.pipelineiq') ALTER COLUMN pain_score COMMENT 'AI-extracted MEDDPICC Pain dimension score (0-10)';
ALTER TABLE IDENTIFIER(:catalog || '.' || :schema || '.pipelineiq') ALTER COLUMN champion_score COMMENT 'AI-extracted MEDDPICC Champion dimension score (0-10)';
ALTER TABLE IDENTIFIER(:catalog || '.' || :schema || '.pipelineiq') ALTER COLUMN implementationplan_score COMMENT 'AI-extracted MEDDPICC Implementation Plan dimension score (0-10)';
ALTER TABLE IDENTIFIER(:catalog || '.' || :schema || '.pipelineiq') ALTER COLUMN decisionprocess_score COMMENT 'AI-extracted MEDDPICC Decision Process dimension score (0-10)';
ALTER TABLE IDENTIFIER(:catalog || '.' || :schema || '.pipelineiq') ALTER COLUMN urgency_score COMMENT 'AI-extracted MEDDPICC Urgency dimension score (0-10)';
ALTER TABLE IDENTIFIER(:catalog || '.' || :schema || '.pipelineiq') ALTER COLUMN competitionawareness_score COMMENT 'AI-extracted MEDDPICC Competition Awareness dimension score (0-10)';
ALTER TABLE IDENTIFIER(:catalog || '.' || :schema || '.pipelineiq') ALTER COLUMN majorblockers_score COMMENT 'AI-extracted MEDDPICC Major Blockers dimension score (0-10)';
ALTER TABLE IDENTIFIER(:catalog || '.' || :schema || '.pipelineiq') ALTER COLUMN measurableimpact_score COMMENT 'AI-extracted MEDDPICC Measurable Impact dimension score (0-10)';
ALTER TABLE IDENTIFIER(:catalog || '.' || :schema || '.pipelineiq') ALTER COLUMN confidence_level_normalized COMMENT 'Normalized confidence level (High, Medium, Low, Not computed)';
ALTER TABLE IDENTIFIER(:catalog || '.' || :schema || '.pipelineiq') ALTER COLUMN next_best_action_recommendation COMMENT 'AI-generated next best action recommendation for account team';
ALTER TABLE IDENTIFIER(:catalog || '.' || :schema || '.pipelineiq') ALTER COLUMN likely_to_slip COMMENT 'Flag if use case is likely to slip (true/false)';
ALTER TABLE IDENTIFIER(:catalog || '.' || :schema || '.pipelineiq') ALTER COLUMN slippage_category COMMENT 'AI-classified primary reason for slippage';
ALTER TABLE IDENTIFIER(:catalog || '.' || :schema || '.pipelineiq') ALTER COLUMN can_be_accelerated COMMENT 'Flag if use case can be accelerated (true/false)';
ALTER TABLE IDENTIFIER(:catalog || '.' || :schema || '.pipelineiq') ALTER COLUMN accel_category COMMENT 'AI-classified best acceleration lever';


In [0]:
%sql
-- Add table properties for tracking AI model and processing metadata
ALTER TABLE IDENTIFIER(:catalog || '.' || :schema || '.pipelineiq') SET TBLPROPERTIES (
  'ai.model.version' = 'databricks-gpt-oss-20b',
  'ai.generation.timestamp' = CURRENT_DATE(),
  'ai.prompt.version' = 'v1.3',
  'pipeline.type' = 'incremental',
  'quality' = 'gold'
);

SELECT 'Metadata added to table: ' || :catalog || '.' || :schema || '.pipelineiq' AS status;


In [0]:
%sql
-- Verify the merge results
SELECT 
  'Total Records' AS metric,
  COUNT(*) AS count
FROM IDENTIFIER(:catalog || '.' || :schema || '.pipelineiq')

UNION ALL

SELECT 
  'Records with AI Enrichment' AS metric,
  COUNT(*) AS count
FROM IDENTIFIER(:catalog || '.' || :schema || '.pipelineiq')
WHERE context IS NOT NULL

UNION ALL

SELECT 
  'Records with Confidence Score' AS metric,
  COUNT(*) AS count
FROM IDENTIFIER(:catalog || '.' || :schema || '.pipelineiq')
WHERE confidence_score IS NOT NULL

UNION ALL

SELECT 
  'High Confidence' AS metric,
  COUNT(*) AS count
FROM IDENTIFIER(:catalog || '.' || :schema || '.pipelineiq')
WHERE confidence_level_normalized = 'High'

UNION ALL

SELECT 
  'Medium Confidence' AS metric,
  COUNT(*) AS count
FROM IDENTIFIER(:catalog || '.' || :schema || '.pipelineiq')
WHERE confidence_level_normalized = 'Medium'

UNION ALL

SELECT 
  'Low Confidence' AS metric,
  COUNT(*) AS count
FROM IDENTIFIER(:catalog || '.' || :schema || '.pipelineiq')
WHERE confidence_level_normalized = 'Low';


In [0]:
%sql
-- Show sample of enriched records (recently processed)
-- Note: To see usecase_name and other source fields, join back to core_usecase_curated on usecase_id
SELECT 
  usecase_id,
  one_liner,
  confidence_score,
  confidence_level_normalized,
  usecase_type,
  business_usecase_type,
  likely_to_slip,
  slippage_category,
  can_be_accelerated,
  accel_category,
  last_modified_date,
  piq_last_updated
FROM IDENTIFIER(:catalog || '.' || :schema || '.pipelineiq')
WHERE piq_last_updated = CURRENT_DATE()
ORDER BY confidence_score DESC
LIMIT 10;


In [0]:
%sql
-- Example: Join back to source table for reporting with full context
-- This query demonstrates how to get source fields with transformations when needed
-- (Or simply use the pipelineiq_reporting view created in the next cell)
-- SELECT 
--   src.usecase_name,
--   src.account_name,
--   CASE WHEN src.field_manager IS NULL THEN 'FLM not assigned' ELSE src.field_manager END AS field_manager,
--   CASE WHEN src.account_solution_architect IS NULL THEN 'SA not assigned' ELSE src.account_solution_architect END AS account_solution_architect,
--   CASE
--     WHEN src.sales_region IS NOT NULL THEN src.sales_region
--     WHEN src.business_unit IS NOT NULL THEN src.business_unit
--     ELSE 'No assigned'
--   END AS sales_region,
--   src.estimated_monthly_dollar_dbus,
--   src.stage,
--   src.target_live_date,
--   piq.one_liner,
--   piq.confidence_score,
--   piq.confidence_level_normalized,
--   piq.usecase_type,
--   piq.business_usecase_type,
--   piq.next_best_action_recommendation,
--   piq.piq_last_updated
-- FROM IDENTIFIER(:catalog || '.' || :schema || '.pipelineiq') piq
-- INNER JOIN main.gtm_data.core_usecase_curated src 
--   ON piq.usecase_id = src.usecase_id
-- WHERE piq.piq_last_updated = CURRENT_DATE()
-- ORDER BY piq.confidence_score DESC
-- LIMIT 10;


In [0]:
%sql
-- Optional: Create a view that joins PipelineIQ with source table for easy reporting
-- This view provides all source fields plus AI enrichments in one place
-- Includes required field transformations
CREATE OR REPLACE VIEW IDENTIFIER(:catalog || '.' || :schema || '.pipelineiq_reporting') AS
SELECT 
  -- Source fields
  src.usecase_id,
  src.usecase_name,
  src.account_name,
  
  -- Transformed fields (apply same logic as original)
  CASE 
    WHEN src.field_manager IS NULL THEN 'FLM not assigned'
    ELSE src.field_manager
  END AS field_manager,
  
  CASE 
    WHEN src.account_solution_architect IS NULL THEN 'SA not assigned'
    ELSE src.account_solution_architect
  END AS account_solution_architect,
  
  CASE
    WHEN src.sales_region IS NOT NULL THEN src.sales_region
    WHEN src.business_unit IS NOT NULL THEN src.business_unit
    ELSE 'No assigned'
  END AS sales_region,
  
  piq.num_of_blockers,
  
  src.account_executive,
  src.estimated_monthly_dollar_dbus,
  src.stage,
  src.target_live_date,
  src.target_onboarding_date,
  src.business_unit,
  src.vertical,
  src.primary_competitor,
  src.target_cloud,
  src.last_modified_date AS source_last_modified_date,
  
  -- PipelineIQ AI enrichments
  piq.piq_last_updated,
  piq.context,
  piq.one_liner,
  piq.next_steps_summary,
  piq.implementation_notes_summary,
  piq.blockers_summary,
  piq.usecase_type,
  piq.business_usecase_type,
  piq.confidence_score,
  piq.confidence_level_normalized,
  piq.rationale_for_confidence_score,
  piq.pain_score,
  piq.champion_score,
  piq.implementationplan_score,
  piq.decisionprocess_score,
  piq.urgency_score,
  piq.competitionawareness_score,
  piq.majorblockers_score,
  piq.measurableimpact_score,
  piq.next_best_action_recommendation,
  piq.likely_to_slip,
  piq.slippage_category,
  piq.can_be_accelerated,
  piq.accel_category
FROM IDENTIFIER(:catalog || '.' || :schema || '.pipelineiq') piq
INNER JOIN main.gtm_data.core_usecase_curated src 
  ON piq.usecase_id = src.usecase_id;

-- Add view metadata
COMMENT ON VIEW IDENTIFIER(:catalog || '.' || :schema || '.pipelineiq_reporting') IS 'Business-facing view of PipelineIQ that joins AI enrichments with source data, including field transformations for reporting and dashboards';

ALTER VIEW IDENTIFIER(:catalog || '.' || :schema || '.pipelineiq_reporting') SET TBLPROPERTIES (
  'ai.pipeline.version' = 'v1.3',
  'ai.model' = 'databricks-gpt-oss-20b',
  'view.purpose' = 'Executive insights and reporting',
  'quality' = 'gold'
);

SELECT 'View created: ' || :catalog || '.' || :schema || '.pipelineiq_reporting' AS status;


In [0]:
%sql
-- View table metadata and properties
DESCRIBE TABLE EXTENDED IDENTIFIER(:catalog || '.' || :schema || '.pipelineiq');


In [0]:
# %sql
# insert into users.sam_lecorre.pipelineiq 
#   SELECT 
#     -- Core identifiers and timestamps
#     usecase_id,
#     last_modified_date,
#     CURRENT_DATE() - interval 1 day AS piq_last_updated,
    
#     -- Fields needed for AI inference (kept for reference)
#     competition_string,
    
#     -- Number of blockers (used in reporting view)
#     num_of_blockers,
    
#     -- AI enriched fields
#     context,
#     one_liner,
#     next_steps_summary,
#     implementation_notes_summary,
#     blockers_summary,
#     usecase_type,
#     business_usecase_type,
    
#     -- Confidence scoring
#     ai_confidence_score_advanced,
#     confidence_score,
#     confidence_level,
#     rationale_for_confidence_score,
#     pain_score,
#     champion_score,
#     implementationplan_score,
#     decisionprocess_score,
#     urgency_score,
#     competitionawareness_score,
#     majorblockers_score,
#     measurableimpact_score,
#     confidence_level_normalized,
    
#     -- Recommendations and classifications
#     next_best_action_recommendation,
#     likely_to_slip,
#     slippage_category,
#     can_be_accelerated,
#     accel_category
#   FROM users.luis_herrera.pipelineiq

In [0]:
%sql
select * from users.luis_herrera.pipelineiq

In [0]:
%sql
select * from users.sam_lecorre.pipelineiq where piq_last_updated = current_date()

In [0]:
%sql
select * from users.luis_herrera.pipelineiq_view where usecase_id in (select usecase_id from users.sam_lecorre.pipelineiq where piq_last_updated = current_date())